control_v11p_sd15_canny [d14c016b]

In [ ]:
import json
import requests
import io
import base64
from PIL import Image, PngImagePlugin

url = "https://3573b85f7cab8e8427.gradio.live"

#open image file using PIL and then covert it to base64 string
# Open the image
# base64_string = ""
# with open('dog.jpg', 'rb') as image_file:
#     base64_bytes = base64.b64encode(image_file.read())
#     print(base64_bytes)

#     base64_string = base64_bytes.decode()

from io import BytesIO

image_path = 'top_wear.png'
image = Image.open(image_path)
buffered = BytesIO()
image.save(buffered, format="PNG")
img_str = base64.b64encode(buffered.getvalue()).decode()

image_path = 'bottom_wear.png'
image = Image.open(image_path)
buffered = BytesIO()
image.save(buffered, format="PNG")
img_str2 = base64.b64encode(buffered.getvalue()).decode()

image_path = 'model2.jpg'
image = Image.open(image_path)
buffered = BytesIO()
image.save(buffered, format="JPEG")
img_str3 = base64.b64encode(buffered.getvalue()).decode()

payload = {
  "denoising_strength": 0.7,
    "prompt": "full-body potrait of female model wearing Casual Puff Sleeves Floral Print Women Pink Top and grey jeans having white background day lighting 2k resolution",
	"negative_prompt": "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck" ,
    "steps": 20 ,
	"alwayson_scripts": {
    "controlnet": {
      "args": [
        {
          "input_image": img_str3, 
		  "module" : "openpose",
      "model" : "control_v11p_sd15_openpose [cab727d4]",
		  "pixel_perfect": True,
      "weight": 0.5,
        },
         {
		  "lowvram": "True",
          "input_image": img_str2, 
		  "module" : "reference_adain+attn",
		  "pixel_perfect": True,
      "threshold_a": 0.5,
      "weight": 1.2,
        },

        {
		  "lowvram": "True",
          "input_image": img_str, 
		  "module" : "reference_adain+attn",
		  "pixel_perfect": True,
      "threshold_a": 0.5,
      "weight": 1.2,
        }
      ]
    }
  }
}

response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)

r = response.json()

print(r)

k = 0
for i in r['images']:
	image = Image.open(io.BytesIO(base64.b64decode(i.split(",",1)[0])))

    # png_payload = {
    #     "image": "data:image/png;base64," + i
    # }
    # response2 = requests.post(url=f'{url}/sdapi/v1/png-info', json=png_payload)

    # pnginfo = PngImagePlugin.PngInfo()
    # pnginfo.add_text("parameters", response2.json().get("info"))
    # image.save('output.png', pnginfo=pnginfo)
	k += 1
	image.save(f'output{k}.png')

In [5]:
import json
import requests
import io
import base64
from PIL import Image, PngImagePlugin

from io import BytesIO

di = []

for i in range(1 , 5):
	image_path = f'{i}.png'
	image = Image.open(image_path)
	buffered = BytesIO()
	image.save(buffered, format="PNG")
	img_str = base64.b64encode(buffered.getvalue()).decode()
	di.append({"image" : image_path , "base64" : img_str})

json_object = json.dumps(di, indent=4)
 
# Writing to sample.json
with open("model.json", "w") as outfile:
    outfile.write(json_object)

In [9]:


url = "https://b7c47e67cb9989e84d.gradio.live"

option_payload = {
    "sd_model_checkpoint": "Realistic_Vision_V5.1",
}

response = requests.post(url=f'{url}/sdapi/v1/options', json=option_payload)
print(response)

<Response [200]>


End to End Test

In [ ]:
query = "Brown T shirt"
import json
import requests
import io
from io import BytesIO
import base64
from PIL import Image, PngImagePlugin

query = "all white outfit with crop top"
res = requests.get("http://127.0.0.1:5000/query", json={"query": query})

for outfit in res.json()["outfits"]:
	print(outfit["top"]["title"])
	print(outfit["bottom"]["title"])
	image = outfit["top"]["image"]
	image = Image.open(BytesIO(base64.b64decode(image.split(",",1)[0])))
	image.show()
	image = outfit["bottom"]["image"]
	image = Image.open(BytesIO(base64.b64decode(image.split(",",1)[0])))
	image.show()